# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
env = UnityEnvironment(file_name="./Banana.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_observation_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 37
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import namedtuple, deque
import random

In [6]:
EPISODES = 2
MAX_STEPS = 200000
EPSILON = 0.5
UPDATE_TARGET_NETWORK_EVERY = 100
REPLAY_BUFFER_SIZE = 1000000
TRAIN_EVERY  = 1
BATCH_SIZE = 32

In [7]:
device = "cpu"

In [8]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples for experience replay"""

    def __init__(self, action_size, buffer_size):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        #self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self,batch_size):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [9]:
class QNetwork(nn.Module):
    def __init__(self,state_size,action_size,hidden_sizes = []):
        super(QNetwork, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        
        assert len(hidden_sizes) > 0
        self.fc1 = nn.Linear(state_size,hidden_sizes[0])
        self.hidden_layers = [] # list of nn.Linear
        
        for i,size in enumerate(hidden_sizes[1:]):
            self.hidden_layers.append(nn.Linear(hidden_sizes[i],size))
            
        self.output_layer = nn.Linear(hidden_sizes[-1],action_size)
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        
        return x

In [10]:
class Agent():
    
    def __init__(self,state_size, action_size):
        print("agent created")
        self.action_size = action_size
        self.q_network_local = QNetwork(state_size,action_size,[50,50]) #network used to make desicion and with online updates
        self.q_network_target = QNetwork(state_size,action_size,[50,50]) #network to calclate the target (copy of local)
        self.q_network_target.eval()
        self.optimizer = optim.Adam(self.q_network_local.parameters(),lr = 0.0001) #TODO:ADD lr as parameter
        
        self.replay_buffer = ReplayBuffer(action_size,REPLAY_BUFFER_SIZE) #TODO: make them variables of constructor
        
    def act(self,state,epsilon=0.0):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        
        random_sampled = np.random.random()
        
        #explore
        if random_sampled <= epsilon:
            action = np.random.randint(action_size)
        else: #exploit
            self.q_network_local.eval()
            with torch.no_grad():
                q_values = self.q_network_local(state)
                
            greedy_action = np.argmax(q_values.cpu().data.numpy())   
            action = greedy_action

        return action
    
    def update_target_network(self):
        for target_param, local_param in zip(self.q_network_target.parameters(), self.q_network_local.parameters()):
            target_param.data.copy_(local_param.data )
            
    def train(self,batch_size):
        (states, actions, rewards, next_states, dones) = self.replay_buffer.sample(batch_size)
        
        
        with torch.no_grad():
            current_target = rewards + 0.99*self.q_network_target(next_states).detach().max(1)[0].unsqueeze(1)
            
        self.q_network_local.train()
        current_estimate =  self.q_network_local(states).gather(1,actions)
        
        mse_loss = F.mse_loss(current_target,current_estimate)
        self.optimizer.zero_grad()
        mse_loss.backward()
        self.optimizer.step()

In [11]:
#env = UnityEnvironment(file_name="./Banana_Linux/Banana.x86_64")
agent = Agent(state_size,action_size)

for episode in range(EPISODES):
    print("start episode ",episode)
    finished = False
    step_count = 0
    episode_score = 0
    
    env_info = env.reset(train_mode=True)[brain_name]
    state = env_info.vector_observations[0]
    
    while not finished and step_count < MAX_STEPS:
        action = agent.act(state,EPSILON) #TODO: decay epsilon
        env_info = env.step(action)[brain_name]
        next_state = env_info.vector_observations[0]
        reward = env_info.rewards[0]
        finished = env_info.local_done[0]
        
        # add experience tuple to experience replay buffer
        agent.replay_buffer.add(state,action,reward,next_state,finished)
        episode_score += reward
        state = next_state
        
        # create a copy of local network to target network
        if step_count % UPDATE_TARGET_NETWORK_EVERY == 0:
            agent.update_target_network
            
            
        if step_count % TRAIN_EVERY == 0 and len(agent.replay_buffer) >=  BATCH_SIZE:
            print("training")
            agent.train(BATCH_SIZE)
            
        step_count += 1
        
    print("Episode reward:",episode_score)

print(len(agent.replay_buffer))
env.close()

agent created
start episode  0
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
training
traini